In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.9 MB/s eta 0:00:00


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm.notebook import tqdm
from PIL import Image
plt.rcParams['figure.figsize'] = [15, 15]

In [2]:

# Read image and convert them to gray!!
def read_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Image not found at path: {path}")

    # Resize to 80% of original size
    height, width = img.shape[:2]
    new_size = (int(width * 0.8), int(height * 0.8))
    img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img_gray, img, img_rgb

In [3]:
def KAZE(img):
    kazeDetector = cv2.KAZE_create()  # Create KAZE detector (supports both AKAZE and KAZE)

    kp, des = kazeDetector.detectAndCompute(img, None)
    return kp, des

def plot_kaze(gray, rgb, kp):
    tmp = rgb.copy()
    img = cv2.drawKeypoints(gray, kp, tmp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img


In [4]:
def matcher(kp1, des1, img1, kp2, des2, img2, threshold):
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < threshold*n.distance:
            good.append([m])

    matches = []
    for pair in good:
        matches.append(list(kp1[pair[0].queryIdx].pt + kp2[pair[0].trainIdx].pt))

    matches = np.array(matches)
    return matches

In [5]:
def homography(pairs):
    rows = []
    for i in range(pairs.shape[0]):
        p1 = np.append(pairs[i][0:2], 1)
        p2 = np.append(pairs[i][2:4], 1)
        row1 = [0, 0, 0, p1[0], p1[1], p1[2], -p2[1]*p1[0], -p2[1]*p1[1], -p2[1]*p1[2]]
        row2 = [p1[0], p1[1], p1[2], 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1], -p2[0]*p1[2]]
        rows.append(row1)
        rows.append(row2)
    rows = np.array(rows)
    U, s, V = np.linalg.svd(rows)
    H = V[-1].reshape(3, 3)
    H = H/H[2, 2] # standardize to let w*H[2,2] = 1
    return H

In [6]:
def random_point(matches, k=4):
    idx = random.sample(range(len(matches)), k)
    point = [matches[i] for i in idx ]
    return np.array(point)

In [7]:
def get_error(points, H):
    num_points = len(points)
    all_p1 = np.concatenate((points[:, 0:2], np.ones((num_points, 1))), axis=1)
    all_p2 = points[:, 2:4]
    estimate_p2 = np.zeros((num_points, 2))
    for i in range(num_points):
        temp = np.dot(H, all_p1[i])
        estimate_p2[i] = (temp/temp[2])[0:2] # set index 2 to 1 and slice the index 0, 1
    # Compute error
    errors = np.linalg.norm(all_p2 - estimate_p2 , axis=1) ** 2

    return errors

In [8]:
def ransac(matches, threshold, iters):
    num_best_inliers = 0

    for i in range(iters):
        points = random_point(matches)
        H = homography(points)

        #  avoid dividing by zero
        if np.linalg.matrix_rank(H) < 3:
            continue

        errors = get_error(matches, H)
        idx = np.where(errors < threshold)[0]
        inliers = matches[idx]

        num_inliers = len(inliers)
        if num_inliers > num_best_inliers:
            best_inliers = inliers.copy()
            num_best_inliers = num_inliers
            best_H = H.copy()

    # print("inliers/matches: {}/{}".format(num_best_inliers, len(matches)))
    return best_inliers, best_H

In [9]:


def stitch_img(left, right, H):
    print("Stitching image ...")

    # Normalize images to float in [0,1]
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Get dimensions of both images
    height_l, width_l, _ = left.shape
    height_r, width_r, _ = right.shape

    # Compute corners for the left image and transform them using H
    corners_left = np.array([[0, 0, 1],
                             [width_l, 0, 1],
                             [width_l, height_l, 1],
                             [0, height_l, 1]]).T  # shape (3,4)
    warped_corners_left = H @ corners_left
    # Normalize homogeneous coordinates:
    warped_corners_left /= warped_corners_left[2, :]

    # Compute corners for the right image (identity transform)
    corners_right = np.array([[0, 0, 1],
                              [width_r, 0, 1],
                              [width_r, height_r, 1],
                              [0, height_r, 1]]).T  # shape (3,4)

    # Combine corners to find overall bounds
    all_x = np.concatenate((warped_corners_left[0, :], corners_right[0, :]))
    all_y = np.concatenate((warped_corners_left[1, :], corners_right[1, :]))

    min_x, max_x = np.min(all_x), np.max(all_x)
    min_y, max_y = np.min(all_y), np.max(all_y)

    # Create a translation matrix to shift all images so that no coordinate is negative
    tx = -min_x if min_x < 0 else 0
    ty = -min_y if min_y < 0 else 0
    translation_mat = np.array([[1, 0, tx],
                                [0, 1, ty],
                                [0, 0, 1]])

    # New canvas size: use ceiling to ensure full coverage
    width_new = int(np.ceil(max_x - min_x))
    height_new = int(np.ceil(max_y - min_y))
    size = (width_new, height_new)

    # Warp left image with the composite transform: translation * H
    warped_left = cv2.warpPerspective(left, translation_mat @ H, size)
    # Warp right image with just the translation matrix (identity warp + translation)
    warped_right = cv2.warpPerspective(right, translation_mat, size)

    # Blend the two warped images.
    # Define a black pixel threshold (all zeros)
    black = np.zeros(3, dtype=np.float32)
    stitch_image = np.zeros_like(warped_left)

    # Iterate over every pixel (this loop might be slow for large images,
    # and vectorized alternatives exist, but here we show the logic clearly)
    from tqdm import tqdm
    for i in tqdm(range(height_new), desc="Blending"):
        for j in range(width_new):
            pixel_left = warped_left[i, j, :]
            pixel_right = warped_right[i, j, :]

            left_valid = not np.allclose(pixel_left, black)
            right_valid = not np.allclose(pixel_right, black)

            if left_valid and not right_valid:
                stitch_image[i, j, :] = pixel_left
            elif right_valid and not left_valid:
                stitch_image[i, j, :] = pixel_right
            elif left_valid and right_valid:
                stitch_image[i, j, :] = (pixel_left + pixel_right) / 2
            # else: remains black

    return stitch_image


In [10]:
import numpy as np

def mask_image_top_left(img_gray, k):
    """
    Keeps the top k% rows and left-most k% columns intact, rest set to black.

    Parameters:
        img_gray (np.ndarray): Grayscale image.
        k (float): Percentage (0 < k <= 100) of image to retain in top and left.

    Returns:
        np.ndarray: Modified image.
    """
    if not (0 < k <= 100):
        raise ValueError("k must be between 0 and 100 (exclusive of 0).")

    h, w = img_gray.shape
    mask = np.zeros_like(img_gray)

    # Calculate cutoff indices
    cutoff_row = int(h * k / 100)
    cutoff_col = int(w * k / 100)

    # Retain top k% rows
    mask[:cutoff_row, :] = img_gray[:cutoff_row, :]

    # Retain left-most k% columns
    mask[:, :cutoff_col] = img_gray[:, :cutoff_col]

    return mask


In [11]:
from PIL import Image
import numpy as np

def detect_black_border_depths(image_pil, black_threshold=10):
    """
    Detects how much black padding exists on each edge of the image separately.

    Args:
        image_pil: PIL.Image object (RGB or grayscale).
        black_threshold: Maximum pixel value considered "black" (0-255).

    Returns:
        (top, bottom, left, right): number of black pixels to crop from each side.
    """
    image_np = np.array(image_pil)

    if image_np.ndim == 3:
        # Convert to grayscale if RGB
        image_np = np.mean(image_np, axis=2)

    h, w = image_np.shape

    # Initialize cropping values
    top = 0
    bottom = 0
    left = 0
    right = 0

    # Detect top
    for y in range(h):
        if np.all(image_np[y, :] <= black_threshold):
            top += 1
        else:
            break

    # Detect bottom
    for y in range(h-1, -1, -1):
        if np.all(image_np[y, :] <= black_threshold):
            bottom += 1
        else:
            break

    # Detect left
    for x in range(w):
        if np.all(image_np[:, x] <= black_threshold):
            left += 1
        else:
            break

    # Detect right
    for x in range(w-1, -1, -1):
        if np.all(image_np[:, x] <= black_threshold):
            right += 1
        else:
            break

    return top, bottom, left, right

def crop_black_borders(image_pil, black_threshold=10):
    """
    Crop black borders independently from each edge.

    Args:
        image_pil: PIL.Image object (RGB or grayscale).
        black_threshold: Maximum pixel value considered "black" (0-255).

    Returns:
        Cropped PIL.Image object.
    """
    top, bottom, left, right = detect_black_border_depths(image_pil, black_threshold)
    w, h = image_pil.size

    # Calculate new box
    left_crop = left
    upper_crop = top
    right_crop = w - right
    lower_crop = h - bottom

    if left_crop >= right_crop or upper_crop >= lower_crop:
        # If everything is cropped away, return original (or could raise an error)
        print("Warning: Cropping would remove entire image. Returning original.")
        return image_pil

    return image_pil.crop((left_crop, upper_crop, right_crop, lower_crop))

In [12]:
def KAZE_matching(img_path1, img_path2, output_dir="./kaze_output", kaze_thresholding=0.7, ransac_thresholding=100, percentage_of_image_used=1.0):
  print(img_path1)
  left_gray, left_origin, left_rgb = read_image(img_path1)
  print(img_path2)
  right_gray, right_origin, right_rgb = read_image(img_path2)

  right_gray = mask_image_top_left(right_gray, percentage_of_image_used * 100)

  cv2.imwrite('right_gray.png', right_gray)

  kp_left, des_left = KAZE(left_gray)
  kp_right, des_right = KAZE(right_gray)

  matches = matcher(kp_left, des_left, left_rgb, kp_right, des_right, right_rgb, 0.7)

  inliers, H = ransac(matches, 100, 2000)

  print(f"We got {len(inliers)} final matches!")

  kaze_stitched = stitch_img(left_rgb, right_rgb, H)

  kaze_stitched_uint8 = (kaze_stitched * 255).astype(np.uint8)
  image_pil = Image.fromarray(kaze_stitched_uint8)

  output_filename = f"0000-{os.path.splitext(os.path.basename(img_path2))[0]}_KAZE_{kaze_thresholding}_{percentage_of_image_used}.jpg"
  output_filepath = os.path.join(output_dir, output_filename)

  cropped_image = crop_black_borders(image_pil, black_threshold=10)
  cropped_image.save(output_filepath, "JPEG", quality=95)

  return output_filename

In [13]:
def stitch_img(left, right, H):
    print("Stitching image ...")

    # Normalize images to float in [0,1]
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Get dimensions of both images
    height_l, width_l, _ = left.shape
    height_r, width_r, _ = right.shape

    # Compute corners for the left image and transform them using H
    corners_left = np.array([[0, 0, 1],
                             [width_l, 0, 1],
                             [width_l, height_l, 1],
                             [0, height_l, 1]]).T  # shape (3,4)
    warped_corners_left = H @ corners_left
    warped_corners_left /= warped_corners_left[2, :]  # Normalize homogeneous coordinates

    # Compute corners for the right image (identity transform)
    corners_right = np.array([[0, 0, 1],
                              [width_r, 0, 1],
                              [width_r, height_r, 1],
                              [0, height_r, 1]]).T  # shape (3,4)

    # Combine corners to find overall bounds
    all_x = np.concatenate((warped_corners_left[0, :], corners_right[0, :]))
    all_y = np.concatenate((warped_corners_left[1, :], corners_right[1, :]))

    min_x, max_x = np.min(all_x), np.max(all_x)
    min_y, max_y = np.min(all_y), np.max(all_y)

    # Create a translation matrix to shift all images so that no coordinate is negative
    tx = -min_x if min_x < 0 else 0
    ty = -min_y if min_y < 0 else 0
    translation_mat = np.array([[1, 0, tx],
                                [0, 1, ty],
                                [0, 0, 1]])

    # New canvas size: use ceiling to ensure full coverage
    width_new = int(np.ceil(max_x - min_x))
    height_new = int(np.ceil(max_y - min_y))
    size = (width_new, height_new)

    # Warp left image with the composite transform: translation_mat @ H
    warped_left = cv2.warpPerspective(left, translation_mat @ H, size)
    # Warp right image with just the translation matrix (identity warp + translation)
    warped_right = cv2.warpPerspective(right, translation_mat, size)

    # Vectorized blending:
    # Create masks where any channel is non-zero (assumed as non-black)
    mask_left = np.any(warped_left != 0, axis=2)
    mask_right = np.any(warped_right != 0, axis=2)

    # Initialize the stitched image as black
    stitch_image = np.zeros_like(warped_left)

    # Pixels only from left image
    only_left = mask_left & ~mask_right
    stitch_image[only_left] = warped_left[only_left]

    # Pixels only from right image
    only_right = mask_right & ~mask_left
    stitch_image[only_right] = warped_right[only_right]

    # Pixels where both images contribute (average the pixel values)
    both = mask_left & mask_right
    stitch_image[both] = (warped_left[both] + warped_right[both]) / 2

    return stitch_image

In [18]:
import os
from tqdm import tqdm

output_path = "Dataset/1_patches_diagonal/"
input_path = "Dataset/1_patches/"

if not os.path.exists(output_path):
  os.mkdir(output_path)

paths = sorted(os.listdir(input_path))

In [19]:
print(paths)

['0000.png', '0001.png', '0002.png', '0003.png', '0004.png', '0005.png', '0006.png', '0007.png', '0008.png', '0009.png', '0010.png', '0011.png', '0012.png', '0013.png', '0014.png', '0015.png', '0016.png', '0017.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0023.png', '0024.png']


In [20]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000
for i in tqdm(range(0, len(paths))):
    print(paths[i - 1], paths[i], sep="\n")
    print(f"0000-{paths[i - 1][:-4]}_KAZE_0.7_1.0.jpg")
    if i == 0:
        continue
    elif i == 1:
        stitched_img_path = KAZE_matching(
            img_path1 = os.path.join(input_path, paths[i - 1]),
            img_path2 = os.path.join(input_path, paths[i]),
            output_dir=output_path,
            kaze_thresholding=0.7,
            ransac_thresholding=100,
            percentage_of_image_used=1.0
        )
        continue

    stitched_img_path = KAZE_matching(
        img_path1 = os.path.join(output_path, f"0000-{paths[i - 1][:-4]}_KAZE_0.7_1.0.jpg"),
        img_path2 = os.path.join(input_path, paths[i]),
        output_dir=output_path,
        kaze_thresholding=0.7,
        ransac_thresholding=100,
        percentage_of_image_used=1.0
    )

  0%|                                                    | 0/25 [00:00<?, ?it/s]

0024.png
0000.png
0000-0024_KAZE_0.7_1.0.jpg
0000.png
0001.png
0000-0000_KAZE_0.7_1.0.jpg
Dataset/1_patches/0000.png
Dataset/1_patches/0001.png


  8%|███▌                                        | 2/25 [00:04<00:50,  2.21s/it]

We got 636 final matches!
Stitching image ...
0001.png
0002.png
0000-0001_KAZE_0.7_1.0.jpg
Dataset/1_patches_diagonal/0000-0001_KAZE_0.7_1.0.jpg
Dataset/1_patches/0002.png


 12%|█████▎                                      | 3/25 [00:05<00:39,  1.81s/it]

...

 20%|████████▊                                   | 5/25 [00:08<00:32,  1.62s/it]

We got 89 final matches!
Stitching image ...
0004.png
0005.png
0000-0004_KAZE_0.7_1.0.jpg
Dataset/1_patches_diagonal/0000-0004_KAZE_0.7_1.0.jpg
Dataset/1_patches/0005.png


 20%|████████▊                                   | 5/25 [00:09<00:38,  1.92s/it]

We got 7 final matches!
Stitching image ...


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:3399: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'
